# Sum & product types, and railways

First you'll practice working with enums and then you can expand the railway example from the lecture.

Have fun!

### Exercise 1

Write the following functions that return options, as there is not always a return value. Don't use any loops, always write them recursively with pattern matching.

1. The function `firstItem` that returns the first item from a list.
2. The function `lastItem` that returns the last item from a list.
3. The function `sumTwoOptions` that sums two options of numbers, there is only a result when both arguments are `Some`.
4. The function `sum` that computes the sum of a list of integers (can be tail recursive).
5. The function `max` that returns the largest item in a list of integers (can be tail recursive).
6. The function `find` that returns the first element that matches. (signature: `('T -> bool) -> 'T list -> 'T Option`)

In [ ]:
let monsters = ["Rathalos"; "Lagiacrus"; "Gore Magala"; "Najarala"]
let numbers = [5;4;3;7;2;10]

let firstItem ls =
    match ls with
        | [] -> None
        | head :: tail -> Some head

let rec lastItem ls =
    match ls with
        | [] -> None
        | [last] -> Some last
        | head :: tail -> lastItem tail 

let sumTwoOptions x y =
    match x with
        | x when x = None || y = None -> None
        | _ -> Some (x.Value + y.Value)

let some5: int Option = Some 5
let some3: int Option = Some 3

let rec sumTail (ls: int List) result = 
    match ls with
        | [] -> None
        | head :: [] -> Some (result + head)
        | head :: tail -> sumTail tail (result + head)

let sumList ls = sumTail ls 0

sumList numbers

let rec maxTail (ls: int list) (current: int Option) =
    match ls with
        | [] when current = None -> None
        | [] -> current
        | head :: tail when head > current.Value ->  maxTail tail (Some head)
        | head :: tail -> maxTail tail current

let max ls = maxTail ls (Seq.tryHead ls)

let rec find boolFunc ls =
    match ls with
        | [] -> None
        | head :: tail when boolFunc head -> Some head
        | head :: tail -> find boolFunc tail

let findNumber x = x = 102





Error: input.fsx (50,15)-(50,22) parse error Successive arguments should be separated by spaces or tupled, and arguments involving function or method applications should be parenthesized
input.fsx (50,23)-(50,30) parse error Successive arguments should be separated by spaces or tupled, and arguments involving function or method applications should be parenthesized

### Exercise 2

Another good use of enums (discriminated unions) is to encode possible states of information. In this exercise you will keep track of contact information. We want to be able to store phone numbers and email addresses. Not everyone needs to have a phone number or an email address, so all four combinations are allowed (neither, only phone, only email, both). 

1. Write one enum (discriminated union) type `ContactInfo` that can store all four states. Note that one does not contain a field, two contain a string and one contains a tuple of two strings.
2. Now we want to add a phone number. Write a function `updatePhone: string -> ContactInfo -> ContactInfo` that adds the phone number to the given ContactInfo instance. It should handle all cases!
3. Do the same for `updateEmail`.
4. Now write a function `sendEmail` that mocks sending an email. This function should only send the email if an email address is known. You can mock sending an email by just printing "sent msg to me@example.com" in the console.
5. Test your implementation

Congrats, you just implementated a simple state pattern!

The enums are very handy to encode the state you are currently in. Then all the state changing functions are simple match functions that return an updated enum.

You can think about how you would use this to for example implement a state pattern for a webshop. The state of the cart could be empty, filled but not paid, and then paid. The mutating functions could add items to the cart or could process payment.

In [ ]:
type ContactInfo = 
    | None
    | Email of String
    | Phone of String
    | EmailPhone of (String * String)

let email = ContactInfo.Email "rathalos@gmail.com"

let updatePhone phoneNumber contactInfo =
    match contactInfo with
        | None -> Phone phoneNumber
        | Email email ->  EmailPhone (email, phoneNumber)
        | Phone _ -> Phone phoneNumber
        | EmailPhone (email, _) -> EmailPhone (email, phoneNumber)

let updateEmail email contactInfo =
    match contactInfo with
        | None -> Email email
        | Email _ -> Email email
        | Phone phone -> EmailPhone (email, phone)
        | EmailPhone (_, phone) -> EmailPhone (email, phone)

let sendEmail contactInfo =
    match contactInfo with
        | None -> printfn "No email adress is known."
        | Email email -> printfn "sent msg to %s" email 
        | Phone _ -> printfn "No email adress is known"
        | EmailPhone (email, _) -> printfn "sent message to %s" email


let newEmail = updateEmail "najarala@gmail.com" email

sendEmail newEmail

sent msg to najarala@gmail.com


### Exercise 3

In the lecture we implemented the `connect` function that makes railway oriented error handling possible. You find this function below, together with mocked versions of `validate`, `updateDb`, and `sendEmail`. You'll expand on this example in two ways:

1. We want to add a prefix to each `request`. This function should take a string `s` and return `"Request" + s`. This function will never produce an error, so it cannot switch to the error track! Think about whether we can still apply `connect` to it and then add the prefix step between `validate` and `updateDb`.
2. Our use case slightly changed, we now always want to send an email, to keep everyone updated even if an error occurs in validation or in the database. However, sending the email can still go wrong. At the end of track we still need a `Result` (`Ok` if the email was sent successfully, `Error` if the email failed). We should never get an `Error` from the database, as we just keep everyone up to date with the email and then it's fine. 
In order to do this we need to be able to switch back from the error track to the happy track. Take a look at the last slide of lecture 4 for the complete track we want to build.
Now write the function `recover` that simply prepends the string with whether the request failed or not. When recover receives `Ok value` it returns the string `"success: value"`, when it receives `Error value` it returns the string `"error: value"`.
Then write the function `catch` to connect `recover` in the pipeline and test the pipeline!

In [ ]:
let rnd = (new System.Random())

let validate (request: string) =
    if request.Length > 5 then Ok request
    else Error "invalid request"

let prefix request =
    "Request " + request

let recover request =
    match request with
        | Ok value -> "success: " + value
        | Error value -> "Error: " + value

let updateDb request =
    let prob = rnd.NextDouble()
    if prob < 0.1 then Error "database unavailable"
    else if prob < 0.3 then Error "request unknown in database"
    else Ok request

let sendEmail request = 
    let prob = rnd.NextDouble()
    if prob < 0.1 then Error "sending email failed"
    else Ok ("email sent successfully (" + request + ")")

let connect func request = 
    match request with
        | Ok value -> func value
        | Error value -> Error value

let connectNoFail func request =
    match request with
        | Ok value -> Ok (func value)
        | Error value -> Error (func value)

let catch func request =
    Ok (func request)



let handleUseCase = validate >> connectNoFail prefix >> connect updateDb >> catch recover >> connect sendEmail

handleUseCase "validRequest"




ResultValue,ErrorValue
email sent successfully (Error: request unknown in database),<null>


When you're all done with this exercise you should think back to JS promises for a moment. You can chain promises together with `then` and recover from errors with `catch`. You might have not used it much, but you can keep going with the chain after a `catch`. 

JS promise chaining is Railway Oriented Programming! 😍

(btw.. I can't get this to work with promises in the notebook, you might have to run it elsewhere if you want to run it.)

In [ ]:
const chain = new Promise((resolve, _) => resolve("value"))

chain
    .then(val => new Promise((_, reject) => reject("oh nee toch niet")))
    .catch(err => new Promise((resolve, _) => resolve("weer opgelost")))
    .then(console.log)